### Пресс-релизы

In [1]:
from pandas import read_csv
texts = read_csv("datasets/politics/china.csv")

In [2]:
print texts.shape

(4393, 6)


In [3]:
doc_titles = []
documents = []

In [4]:
#doc_titles = []
for i in range(texts.shape[0]):
    if isinstance(texts['title'][i], float):
        doc_titles.append('')
    else:
        doc_titles.append('_'.join(texts['title'][i].split()))

In [5]:
#documents = []

for i in range(texts.shape[0]):
    if not isinstance(texts['contents'][i], float):
        documents.append(texts['contents'][i].split())  
        if isinstance(texts['title'][i], float):
            doc_titles.append('some_title')
        else:
            doc_titles.append('_'.join(texts['title'][i].split()))
    
# print documents[0]

### ММРО

In [28]:
import h5py
import numpy as np

f = h5py.File("data.hdf")

In [29]:
for name in f:
    print name

dictionary
documents
metadata
n_wd_coo
p_td
p_wt


In [7]:
print f['documents/0'][:]

[ (420, 258, 266, [30, 0, 49, 25, 2], [0.6000718474388123, 0.20639878511428833, 0.1268741488456726, 0.04164036735892296, 0.015201207250356674], [0.20639878511428833, 0.6000718474388123, 0.1268741488456726, 0.015201207250356674, 0.04164036735892296, 0.00773442629724741, 0.0006645901594310999, 0.0014147062320262194, 0.0])
 (12516, 267, 280, [30, 0, 23, 14, 2], [0.7380298972129822, 0.22457295656204224, 0.01656961999833584, 0.009636436589062214, 0.0063199447467923164], [0.22457295656204224, 0.7380298972129822, 0.00018846068996936083, 0.0063199447467923164, 0.0, 0.004682679660618305, 0.01656961999833584, 0.009636436589062214, 1.5038111224471365e-19])
 (9276, 282, 292, [0, 30, 49, 2, 14], [0.736145555973053, 0.1370600461959839, 0.09295647591352463, 0.014059280976653099, 0.006313590332865715], [0.736145555973053, 0.1370600461959839, 0.09295647591352463, 0.014059280976653099, 0.0050178165547549725, 0.003167149843648076, 0.005280101206153631, 0.006313590332865715, 8.658378128061394e-19])
 (1103

In [8]:
doc_titles = []
for name in f['metadata']:
    doc_titles.append(name[0])

In [9]:
dictionary = f['dictionary'][:]
print len(dictionary)

17242


In [34]:
print f['dictionary']

<HDF5 dataset "dictionary": shape (17242,), type "|O8">


In [27]:
metadata = f['metadata'][:]
for i in metadata[0]:
    print i
    

2007-MMRO13/Dedovets
Алгоритм распознавания, основанный на построении метрических закономерностей
Дедовец М.С., Сенько О.В.
дедовец07алгоритм
MMRO
13
2007


In [10]:
documents = []
docs_number = len(f['documents'])
for i in range(docs_number):
    documents.append([dictionary[token[0]] for token in f['documents/{}'.format(i)][:]])

In [14]:
print doc_titles[0]
for word in documents[0]:
    print word,    

2007-MMRO13/Dedovets
алгоритм распознавание основать построение метрический закономерность алгоритм распознавание основать построение метрический закономерность сенький ранее быть предложить связанный решение задача распознавание средство анализ дать основать поиск закономерность закономерность пониматься подобласть признаковый пространство содержимый объект один класс полный закономерность преимущественно один класс частичный закономерность геометрический форма подобласть определяться заранее задать модель наиболее часто использовать модель являться модель логический закономерность который искомое область признаковый пространство иметь форма гиперпараллелепипед рассматриваться закономерность формировать помощь линейный граница произвольный ориентация координатный ось несомненно априорный задание геометрический форма являться существенный ограничение затруднять выявление закономерность реально существующий дать удовлетворять сделать предположение связь предложный новый закономерность к

### Фильтрация словаря

In [6]:
# count word frequencies
from collections import defaultdict
freq = defaultdict(int)

for document in documents:
    for word in document:
        freq[word] += 1
        
#for (word, count) in sorted(freq.items(), key=lambda x:x[1]):
#    print word, count

In [7]:
# choose thresholds 

# usa
#no_less = 4
#no_more = 48000

# china
no_less = 4
no_more = 10000

# MMRO
#no_less = 4
#no_more = 9905

In [8]:
# build vocabulary and vector representation
word_to_index = {}
index_to_word = {}
vectors = []
for document in documents:
    vectors.append([])
    for word in document:
        if freq[word] >= no_less and freq[word] <= no_more:
            if word not in word_to_index:
                index = len(word_to_index)
                word_to_index[word] = index
                index_to_word[index] = word
            vectors[-1].append(word_to_index[word])   
if len(vectors) != len(documents):
    print "ATTENTION! SOME DOCUMENTS VANISHED!"

In [9]:
# save vocab
path = "datasets/politics/vocab.china.txt"

import codecs
with codecs.open(path, "w", encoding='utf-8') as fout:
    for (index, word) in sorted(index_to_word.items()):
        print>>fout, word

In [10]:
print len(index_to_word)

8904


In [11]:
# save vowpal wabbit
path = "datasets/politics/vw.china.txt"

import codecs
with codecs.open(path, "w", encoding='utf-8') as fout:
    for (num, vector) in enumerate(vectors):
        print>>fout, doc_titles[num],
        for index in vector:
            print>>fout, index_to_word[index],
        print>>fout

In [11]:
# comute bow
bow = np.zeros((len(vectors), len(index_to_word)))
for (docnum, vector) in enumerate(vectors):
    for index in vector:
        bow[docnum, index] += 1
        
print bow.shape

(1061, 7805)


In [12]:
# save bow
path = "docword.mmro.txt"

import codecs
with codecs.open(path, "w", encoding='utf-8') as fout:
    print>>fout, bow.shape[0]
    print>>fout, bow.shape[1]
    print>>fout, np.sum(bow != 0)
    
    empty_d = 1
    for d in range(bow.shape[0]):
        if bow[d, :].sum() < 1:
            print 'haha'
            continue
        for w in range(bow.shape[1]):
            if bow[d, w] > 0:
                print>>fout, d + 1, w + 1, int(bow[d, w])

### Подсчет величин n(u, v) и n(w)

In [12]:
# occ and cooc based on DOCUMENTS
from collections import defaultdict
cooc = defaultdict(int)
occ = defaultdict(int)

W = len(index_to_word)
for document in vectors:
    seen = defaultdict(bool)
    for word in document:
        if word not in seen:
            for friend in seen:
                cooc[min(word, friend) * W + max(word, friend)] += 1
            occ[word] += 1
            seen[word] = True

norm = len(vectors)

In [9]:
# occ and cooc based on WINDOWS (is this interpretation reasonable?)
k = 100

from collections import defaultdict
cooc = defaultdict(int)
occ = defaultdict(int)
lastpos = defaultdict(lambda: -1)
windows = 0
W = len(index_to_word)

for document in vectors:
    length = len(document)
    windows += length - k + 1
    for (rightpos, right) in enumerate(document):
        #print 'right ', rightpos, right, index_to_word[right]
        if min(rightpos, length - k) > max(lastpos[right], rightpos - k):
            occ[right] += min(rightpos, length - k) - max(lastpos[right], rightpos - k) 
            #print 'added ', min(rightpos, length - k) - max(lastpos[right], rightpos - k)  
        
        for leftpos in range(max(0, rightpos - k + 1), rightpos):
            left = document[leftpos]
            #print 'left ', leftpos, left, index_to_word[left]
            if leftpos >= lastpos[left] and min(leftpos, length - k) > max(lastpos[right], rightpos - k):
                cooc[min(left, right) * W + max(left, right)] += min(leftpos, length - k) - max(lastpos[right], rightpos - k) 
                #print 'added ', leftpos - max(lastpos[right], rightpos - k) 

        lastpos[right] = rightpos         

norm = windows

In [10]:
print norm

1246289


### Составление формулы и сохранение матрицы

In [11]:
# условная вероятность p(нашего слова| чужого)
import codecs
counter = 0
with codecs.open("condprob_w100.mmro.txt", "w", encoding='utf-8') as fout:
    for key in cooc:
        words = divmod(key, W)
        for i in range(2):
            j = 1 - i
            cond_prob = 1.0 * cooc[key] / occ[words[j]]
            counter += 1
            print>>fout, words[i], words[j], cond_prob #, cooc[key], occ[words[j]], index_to_word[words[i]], index_to_word[words[j]]
print counter

7458662


In [11]:
# positive PMI [Newman]
import numpy as np
import codecs
with codecs.open("datasets/politics/pPMI_w100_china.txt", "w", encoding='utf-8') as fout:
    for key in cooc:
        words = divmod(key, W)
        PMI = np.log(1.0 * norm * cooc[key] / occ[words[0]] / occ[words[1]])
        if PMI > 0:
            print>>fout, words[0], words[1], PMI #, cooc[key], occ[words[0]], occ[words[1]], index_to_word[words[0]], index_to_word[words[1]]

ZeroDivisionError: float division by zero

In [11]:
# логарифмы [Mimno]

import codecs
with codecs.open("mimno.mmro.comments.txt", "w", encoding='utf-8') as fout:
    counter = 0
    for key in cooc:
        counter += 1
        if counter > 1000:
            break
        words = divmod(key, W)
        for i in range(2):
            j = 1 - i
            mimno = np.log((cooc[key] + 1.0) / occ[words[i]])
            print>>fout, words[i], words[j], mimno, cooc[key], occ[words[i]], index_to_word[words[i]], index_to_word[words[j]]

In [ ]:
# логарифмы с tf-idf [Кольцов]